In [2]:
import os.path
import pandas as pd
from unidecode import unidecode
from utils import Wrangling

In [3]:
# check if all_data.csv exists, if it doesn't, run function to create it:
if os.path.exists("all_data.csv"):
    data = pd.read_csv("all_data.csv", dtype = str)
else:
    Wrangling.save_tables_from_pdfs()
    data = pd.read_csv("all_data.csv", dtype = str)

In [ ]:
data_problem = pd.read_csv("all_data.csv", dtype = str) ## we have issue with tables that share multiple ministerios and same dates... check 20220218_135419.pdf. These files will be removed
data_problem["file_name"] = data_problem["file_name"].astype(str)
data_problem = data_problem[~data_problem["file_name"].str.contains("pdf")]

# we could potentially keep these:
# data_problem["previsao_de_passageiros"] = data_problem["previsao_de_passageiros"].astype(str)
# data_problem = data_problem[~data_problem["previsao_de_passageiros"].str.contains("pdf")]

pd.DataFrame(data_problem).to_csv("fix.csv") # save for checking

# keep only "good quality" data:
data["file_name"] = data["file_name"].astype(str)
data = data[data["file_name"].str.contains("pdf")]

cols = ["autoridades_apoiadas","origem","decolagem_h_local","destino","pouso_h_local","motivo","previsao_de_passageiros","file_name"]
data = data[cols] # select specific cols

# Convert NaN values to empty string
nan_value = float("NaN")

data.replace("", nan_value, inplace=True)
data.dropna(subset = ["decolagem_h_local"], inplace=True)

# pd.DataFrame(data).to_csv("clean.csv") # save for checking
data["origem"] = data['origem'].str.replace(r"\(.*\)","")
data["origem"] = data['origem'].str.strip()
data["origem"] = data["origem"].apply(unidecode)

g = data.groupby('origem').nunique().reset_index()

# pd.DataFrame(g).to_csv("summary.csv") # save for checking

airports = pd.read_csv("airports.csv")

cols_names = ["AirportID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST", "TZ_DB", "Type", "Source"]
airports.columns = cols_names

# to lower
airports["City"] = airports["City"].str.lower()
# keep only cols of interest:
airports = airports[["City","Country","Latitude","Longitude"]]
g = g.merge(airports, left_on='origem', right_on='City', how="left")

pd.DataFrame(pd.DataFrame(g.groupby(["origem", "Country"]).head(1)).to_csv("merge_good.csv")) # save for checking

# g[g['ids'].str.contains("ball")]
g1 = g.loc[g['Country'].isna()]
print(g1)
#pd.DataFrame(pd.DataFrame(g1).to_csv("merge_bad.csv")) # save for checking


# print(g)
# explore and groom data:
## unique columns
## clean df
## summary origem and destino
### might help: df.sort_values(by='value').groupby('date').head(2)
## fix column types - number, date and str 

# for df in dfs:
#     clean_df = clean_columns(df)
#     print(clean_df.columns)


# summary table
# https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
# docs: https://openflights.org/data.html
# https://github.com/kelvins/Municipios-Brasileiros/blob/main/csv/municipios.csv
# https://www.trilhaseaventuras.com.br/siglas-dos-principais-aeroportos-do-brasil-iata/
# https://pt.wikipedia.org/wiki/Lista_de_aeroportos_do_Brasil_por_c%C3%B3digo_aeroportu%C3%A1rio_ICAO

# codigo uf:
# https://www.oobj.com.br/bc/article/quais-os-c%C3%B3digos-de-cada-uf-no-brasil-465.html

## como as viagens mudaram - total, nacional/ internacional por mandatos
## qual ministerio viajou mais? por mandato?
## overlap routes
### https://towardsdatascience.com/mapping-the-worlds-flight-paths-with-python-232b9f7271e5